In [23]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math
import os
import pyarrow.feather as feather
from lightgbm import LGBMClassifier, log_evaluation
from sklearn.model_selection import StratifiedKFold
from sklearn import preprocessing
from colorama import Fore, Back, Style
import optuna.integration.lightgbm as lgb
import optuna

import warnings
warnings.filterwarnings("ignore")

In [2]:
#skip make_data
data_use=pd.read_feather("../Kaggle/AMEX/train_use.ftr")
train_labels=pd.read_csv("../Kaggle/AMEX/train_labels.csv")
print(data_use.shape)
print(train_labels.shape)

def categorization(data):

    num_cols=data._get_numeric_data().columns
    cat_cols=list(set(data.columns) - set(num_cols))

    for column in cat_cols:
        target_column = data[column]
        le = preprocessing.LabelEncoder()
        le.fit(target_column)
        label_encoded_column = le.transform(target_column)
        data[column] = pd.Series(label_encoded_column).astype('category')
    
    return data

data_use=categorization(data_use)

(458913, 2033)
(458913, 2)


In [3]:
data_use=data_use.iloc[0:1000,0:40]
train_labels=train_labels.iloc[0:1000,:]


In [19]:
#lightGBM
def amex_metric(y_true: np.array, y_pred: np.array) -> float:

    y_true=np.array(y_true)
    y_pred=np.array(y_pred)

    # count of positives and negatives
    n_pos = y_true.sum()
    n_neg = y_true.shape[0] - n_pos

    # sorting by descring prediction values
    indices = np.argsort(y_pred)[::-1]
    preds, target = y_pred[indices], y_true[indices]

    # filter the top 4% by cumulative row weights
    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_filter = cum_norm_weight <= 0.04

    # default rate captured at 4%
    d = target[four_pct_filter].sum() / n_pos

    # weighted gini coefficient
    lorentz = (target / n_pos).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    # max weighted gini coefficient
    gini_max = 10 * n_neg * (1 - 19 / (n_pos + 20 * n_neg))

    # normalized weighted gini coefficient
    g = gini / gini_max

    return 0.5 * (g + d)

def lgb_amex_metric(y_true, y_pred):
    """The competition metric with lightgbm's calling convention"""
    return ('amex',
            amex_metric(y_true, y_pred),
            True)

In [5]:
def amex_metric_optuna(preds: np.ndarray, data: lgb.Dataset) -> float:

    y_pred = preds
    y_true = data.get_label()

    # count of positives and negatives
    n_pos = y_true.sum()
    n_neg = y_true.shape[0] - n_pos

    # sorting by descring prediction values
    indices = np.argsort(y_pred)[::-1]
    preds, target = y_pred[indices], y_true[indices]

    # filter the top 4% by cumulative row weights
    weight = 20.0 - target * 19.0
    cum_norm_weight = (weight / weight.sum()).cumsum()
    four_pct_filter = cum_norm_weight <= 0.04

    # default rate captured at 4%
    d = target[four_pct_filter].sum() / n_pos

    # weighted gini coefficient
    lorentz = (target / n_pos).cumsum()
    gini = ((lorentz - cum_norm_weight) * weight).sum()

    # max weighted gini coefficient
    gini_max = 10 * n_neg * (1 - 19 / (n_pos + 20 * n_neg))

    # normalized weighted gini coefficient
    g = gini / gini_max

    return "AMEX score", 0.5 * (g + d), True

In [25]:
def fit_lgbm(trial, train, val, devices=(-1,),  cat_features=None, num_rounds=1500):
    """Train Light GBM model"""
    X_train, y_train = train
    X_valid, y_valid = val
    
    
    params = {
        #type
        'objective': 'binary',
        "boosting": "gbdt",
        "metric": "AMEX score",
        "verbose": -1,
        #tree structure
        'num_leaves': trial.suggest_int('num_leaves', 2, 300),
        'max_depth': trial.suggest_int('max_depth', 2, 12),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 400),
        #accuracy
        "learning_rate": 0.1,
        #regularization
        'lambda_l1': trial.suggest_float('lambda_l1', 0, 0.98),
        'lambda_l2': trial.suggest_float('lambda_l1', 0, 0.98),
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0, 15),
        #bagging
        "bagging_freq": 1,
        "bagging_fraction": trial.suggest_float('bagging_fraction', 0, 0.98),
        "feature_fraction": trial.suggest_float('feature_fraction', 0, 0.98),
        'seed':123,
    }

    device = devices[0]
    if device == -1:
        # use cpu
        pass
    else:
        # use gpu
        print(f'using gpu device_id {device}...')
        params.update({'device': 'gpu', 'gpu_device_id': device})


    d_train = lgb.Dataset(X_train, label=y_train, categorical_feature=cat_features)
    d_valid = lgb.Dataset(X_valid, label=y_valid, categorical_feature=cat_features)
    watchlist = [d_train, d_valid]
    
    #pruning_callback = optuna.integration.LightGBMPruningCallback(trial, 'auc', valid_name='valid_1')  

    FIT_PARAMS_LGB = {"num_boost_round": 10000, "early_stopping_rounds": 10, "verbose_eval":0}

    callbacks = [
    lgb.log_evaluation(0),
    lgb.early_stopping(10)]

    model = lgb.train(params, **FIT_PARAMS_LGB,
                categorical_feature=cat_features,
                train_set=d_train,
                valid_sets=watchlist,
                callbacks=callbacks, # コマンドライン出力用コールバック関数
                feval=amex_metric_optuna,
                     )

    # predictions
    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)
    
    #print('best_score', model.best_score)
    log = {'valid/amex': amex_metric(y_valid, y_pred_valid)}
    return model, y_pred_valid, log

In [7]:
def objective(trial, fast_check=True, return_info=False):
        folds = 5
        seed = 666
        shuffle = True
        kf = StratifiedKFold(n_splits=folds, shuffle=shuffle, random_state=seed)
    
        y_valid_pred_total = np.zeros(data_use.shape[0])
        models = []
        valid_score = 0

        for train_idx, valid_idx in kf.split(data_use, train_labels["target"]):
            train_data = data_use.iloc[train_idx,:], train_labels["target"].iloc[train_idx]
            valid_data = data_use.iloc[valid_idx,:], train_labels["target"].iloc[valid_idx]
            
            #print('train', len(train_idx), 'valid', len(valid_idx))
    
            model, y_pred_valid, log = fit_lgbm(trial, train_data, valid_data, num_rounds=1200)
            
            y_valid_pred_total[valid_idx] = y_pred_valid
            models.append(model)
            
            valid_score += log["valid/amex"]
            if fast_check:
                break
                
        valid_score /= len(models)
        
        if return_info:
            return valid_score, models, y_valid_pred_total, train_labels["target"]
        else:
            return valid_score
   

In [27]:
optuna.logging.set_verbosity(optuna.logging.ERROR)
study = optuna.create_study()
study.optimize(objective, timeout=60*0.5)

train 800 valid 200


feature_fraction, val_score: 0.079425:  14%|#4        | 1/7 [00:00<00:00, 15.10it/s]

[LightGBM] [Warning] Unknown parameter: score
[LightGBM] [Warning] min_data_in_leaf is set=311, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=311
Training until validation scores don't improve for 10 rounds


num_leaves, val_score: 0.079425:  20%|##        | 4/20 [00:00<00:00, 17.06it/s]

Early stopping, best iteration is:
[1]	valid_0's AMEX score: 0.0392608	valid_1's AMEX score: 0.0794247


...

Early stopping, best iteration is:
[32]	valid_0's AMEX score: 0.692095	valid_1's AMEX score: 0.494764
Early stopping, best iteration is:
[32]	valid_0's AMEX score: 0.692095	valid_1's AMEX score: 0.494764


bagging, val_score: 0.429015: 100%|##########| 10/10 [00:00<00:00, 10.33it/s]


Early stopping, best iteration is:
[32]	valid_0's AMEX score: 0.692095	valid_1's AMEX score: 0.494764


min_data_in_leaf, val_score: 0.404988: 100%|##########| 5/5 [00:00<00:00, 12.86it/s]

best_score defaultdict(<class 'collections.OrderedDict'>, {'valid_0': OrderedDict([('AMEX score', 0.5263168550862001)]), 'valid_1': OrderedDict([('AMEX score', 0.40498840957362614)])})


In [11]:


params_last = {
        #type
        'objective': 'binary',
        "boosting": "gbdt",
        "metric": "AMEX score",
        "verbose": -1,
        #tree structure
        'num_leaves': 33,
        'max_depth': 5,
        'min_data_in_leaf': 25,
        #accuracy
        "learning_rate": 0.1,
        #regularization
        'lambda_l1': 0.0009192401584865857,
        'lambda_l2': 0.011545006628700807,
        'min_gain_to_split': 3,
        #bagging
        "bagging_freq": 1,
        "bagging_fraction": 0.46461405788976207,
        "feature_fraction": 0.5,
        'seed':123,
    }

In [13]:
#valid_score, models0, y_pred_valid, y_train = objective(optuna.trial.FixedTrial(study.best_params), fast_check=False, return_info=True)
valid_score, models0, y_pred_valid, y_train = objective(optuna.trial.FixedTrial(params_last), fast_check=False, return_info=True)

[I 2022-06-23 08:24:06,590] A new study created in memory with name: no-name-b124e6fe-b7e8-4b2a-aaa7-300d00ec12b0


train 800 valid 200
training LGB:


feature_fraction, val_score: inf:   0%|          | 0/7 [00:00<?, ?it/s]/Users/rikuta/miniforge3/envs/env/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/rikuta/miniforge3/envs/env/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
feature_fraction, val_score: 0.549688:  14%|#4        | 1/7 [00:00<00:00,  9.48it/s]

[LightGBM] [Warning] Unknown parameter: score
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
Training until validation scores don't improve for 10 rounds
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[10]	valid_0's AMEX score: 0.730371	valid_1's AMEX score: 0.549688
Training until validation scores don't improve for 10 rounds


/Users/rikuta/miniforge3/envs/env/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/rikuta/miniforge3/envs/env/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
feature_fraction, val_score: 0.549688:  29%|##8       | 2/7 [00:00<00:00,  7.36it/s]/Users/rikuta/miniforge3/envs/env/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed 

...

/Users/rikuta/miniforge3/envs/env/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/rikuta/miniforge3/envs/env/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
min_data_in_leaf, val_score: 0.525399:  80%|########  | 4/5 [00:00<00:00, 11.97it/s]

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's AMEX score: 0.645585	valid_1's AMEX score: 0.525399


[I 2022-06-23 08:24:12,528] Trial 63 finished with value: 0.5253994459628332 and parameters: {'min_child_samples': 100}. Best is trial 60 with value: 0.5253994459628332.
min_data_in_leaf, val_score: 0.525399:  80%|########  | 4/5 [00:00<00:00, 11.97it/s]/Users/rikuta/miniforge3/envs/env/lib/python3.8/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/rikuta/miniforge3/envs/env/lib/python3.8/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's AMEX score: 0.645585	valid_1's AMEX score: 0.525399
best_score defaultdict(<class 'collections.OrderedDict'>, {'valid_0': OrderedDict([('AMEX score', 0.6455854468508382)]), 'valid_1': OrderedDict([('AMEX score', 0.5253994459628332)])})


KeyError: '[105, 154, 14, 191, 29, 10, 18, 74, 158, 162, 188, 3, 42, 87, 25, 100, 122, 4, 33, 138, 160, 126, 133, 75, 60, 58, 140, 24, 169, 2, 94, 21, 135, 117, 52, 128, 92, 19, 43, 69, 81, 145, 13, 198, 66, 118, 182, 193, 35, 116, 164, 132, 67, 166, 86, 22, 23, 111, 172, 88, 175, 143, 178, 93, 65, 70, 68, 150, 121, 152, 147, 190, 196, 192, 64, 59, 56, 16, 12, 11, 9, 20, 63, 7, 45, 44, 26, 46, 47, 41, 40, 49, 50, 51, 39, 38, 53, 54, 37, 31, 199, 99, 71, 170, 137, 139, 142, 148, 149, 151, 156, 159, 161, 163, 165, 167, 173, 134, 174, 177, 179, 180, 183, 185, 186, 187, 189, 194, 195, 197, 136, 72, 102, 73, 77, 78, 79, 80, 82, 83, 84, 85, 96, 98, 101, 130, 104, 107, 108, 112, 113, 115, 119, 123, 124, 125, 127, 129, 0] not in index'

In [ ]:
num_cols=data_use._get_numeric_data().columns
cat_features=list(set(data_use.columns) - set(num_cols))    


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1)


#timeout = 60 * 30

#study = optuna.create_study(
#    pruner=optuna.pruners.SuccessiveHalvingPruner(min_resource=2, reduction_factor=4, min_early_stopping_rate=1))
#study.optimize(objective, timeout=timeout)

In [40]:
study.best_params

ValueError: No trials are completed yet.

In [223]:
#ensemble
test=pd.read_feather("../Kaggle/AMEX/test_data.ftr")
sub1 = pd.read_csv('../Kaggle/AMEX/submission_ens.csv')

In [278]:
#CatBoost prediction
col=data_use.columns
test_ID=test['customer_ID'].unique()
pred_catboost= clf.predict_proba(test_use[col])[:, 1]

In [280]:
sub2= pd.DataFrame({'customer_ID':test_ID, 'prediction2':pred_catboost})

In [283]:
#Ensemble
sub1 = pd.read_csv('../Kaggle/AMEX/submission_ens.csv')

blend = pd.merge(sub1, sub2, how='inner', on='customer_ID')
blend.prediction = (blend.prediction * 0.95 + blend.prediction2 * 0.05)
blend[['customer_ID', 'prediction']].to_csv('../Kaggle/AMEX/0613submission2.csv', index=False)

In [ ]:
#lightgbm fit
ONLY_FIRST_FOLD = True
INFERENCE = True


model=lgb.train(params, 
                categorical_feature=cat_features,
                train_set=d_train,
                valid_sets=watchlist,
                num_boost_round=10000,  # 最大学習サイクル数。early_stopping使用時は大きな値を入力
                callbacks=callbacks, # コマンドライン出力用コールバック関数
                feval=amex_metric_optuna,
                )
      
score_list = []
y_pred_list = []
kf = StratifiedKFold(n_splits=5)
for fold, (idx_tr, idx_va) in enumerate(kf.split(data_use, train_labels["target"])):
    X_tr, X_va, y_tr, y_va, model = None, None, None, None, None

    X_tr = data_use.iloc[idx_tr]
    X_va = data_use.iloc[idx_va]
    y_tr = train_labels["target"][idx_tr]
    y_va = train_labels["target"][idx_va]
    
    model = my_booster()

    model.fit(X_tr, y_tr,
            eval_set = [(X_va, y_va)], 
            eval_metric=[lgb_amex_metric],
            callbacks=[log_evaluation(100)])
    X_tr, y_tr = None, None
    y_va_pred = model.predict_proba(X_va, raw_score=True)
    score = amex_metric(y_va.to_numpy(), y_va_pred)
    n_trees = model.best_iteration_
    if n_trees is None: n_trees = model.n_estimators
    print(f"{Fore.GREEN}{Style.BRIGHT}Fold {fold} "
          f"                Score = {score:.5f}{Style.RESET_ALL}")
    score_list.append(score)
    
    if INFERENCE:
        y_pred_list.append(model.predict_proba(test, raw_score=True))
        
    if ONLY_FIRST_FOLD: break # we only want the first fold
    
print(f"{Fore.GREEN}{Style.BRIGHT}OOF Score:   {np.mean(score_list):.5f}{Style.RESET_ALL}")